In [1]:

import json
import os
from weakref import ref
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import shutil
import matplotlib.image
from glob import glob
import torch

In [2]:
experiment_dir="/home/wangdong/autoencoder_denoiser/experiment_data/t1_noise/"
state_dict = torch.load(os.path.join(experiment_dir, 'latest_model.pt'))
%run model_factory
model = UNet(1,1,True)


In [3]:
# 
model.load_state_dict(state_dict['model'])
model = torch.nn.DataParallel(model)

In [4]:
import cv2
orig_img_dir = "/home/wangdong/autoencoder_denoiser/dataset/real_noise"
new_img_dir = orig_img_dir+"_binary_array"

clist = [(0,"green"), (0.25,"white"), (0.75, "black"), (1, "red")]
# red means left real signal
custom_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("_",clist)


        

In [6]:
model.eval()
count = 0
torch.cuda.empty_cache()
for img_folder in glob(new_img_dir+"/*/"):
    for img_path in glob(img_folder+"/*"):
        count+=1
        img = np.load(img_path)
        # plt.imshow(img, cmap="Greys")
        img_input = torch.tensor(img).unsqueeze(0).unsqueeze(0).float().cpu()
        denoised = model.forward(img_input)
        denoised = torch.clip(denoised.round(),0,1)
        denoised_result = denoised[0,0].cpu().detach().numpy()
        difference = img*3 - denoised_result 
        plt.imshow(difference, cmap = custom_cmap, vmax=3, vmin=-1)
        plt.savefig(f"/home/wangdong/autoencoder_denoiser/denoised_real_pics/{count}.png")
        del img_input,denoised
        torch.cuda.empty_cache()
        if count > 30 : 
            break
    
        

RuntimeError: CUDA out of memory. Tried to allocate 452.00 MiB (GPU 0; 23.70 GiB total capacity; 21.56 GiB already allocated; 202.56 MiB free; 21.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.cuda.empty_cache()